In [ ]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# **** read data

In [ ]:
## read data
train_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
sample_submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
y = train_data['SalePrice']
y.shape

In [ ]:
describe_data = train_data.describe()
describe_data

In [ ]:
##show data
train_data.head(10)

In [ ]:
##histogram
##draw the oytput "y" 

sns.distplot(y);

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

# 1.**Draw the Relationship between the numerical variable input and autput

In [ ]:
##Draw the relationship between "LotArea" and the output
z = train_data['LotArea']
print(z.shape)
plt.scatter(z,y)

In [ ]:
##Draw the relationship between "TotalBsmtSF" and the output
p = train_data['TotalBsmtSF']
print(z.shape)
plt.scatter(p,y)


In [ ]:
##Draw the relationship between "1stFlrSF" and the output
c = train_data['1stFlrSF']
print(z.shape)
plt.scatter(c,y)

In [ ]:
##Draw the relationship between "GarageArea" and the output
u = train_data['GarageArea']
print(z.shape)
plt.scatter(u,y)

In [ ]:
categorical_features = train_data.select_dtypes([object]).columns
numerical_features = train_data.select_dtypes([int, float]).columns

In [ ]:
fig = plt.figure(figsize=(25,40)) #figure size
o = 13  # number of rows
q = 3  # number of columns
w = 1  # initialize plot counter

for feat in numerical_features:
    plt.subplot(o, q, w)
    sns.kdeplot(x=train_data[feat])
    w+=1
    
plt.tight_layout()
plt.show()

In [ ]:
#histogram and normal probability plot
sns.distplot(y, fit=norm);
fig = plt.figure()
res = stats.probplot(y, plot=plt)

In [ ]:
y  = np.log(y)

In [ ]:
##transformed histogram and normal probability plot
sns.distplot(y, fit=norm);
fig = plt.figure()
res = stats.probplot(y, plot=plt)

 # ********missing data 

In [ ]:
## delaing measing data in train_data
train_data.isna().sum()[train_data.isna().sum()>0]
train_data.fillna('Unknown',inplace=True)
print(train_data.shape)

In [ ]:
train_data.isnull().sum()

In [ ]:
## delaing measing data in test_data
test_data.isna().sum()[test_data.isna().sum()>0]
test_data.fillna('Unknown',inplace=True)
print(test_data.shape)

In [ ]:
test_data.isnull().sum()

# ****convert data from string to number

In [ ]:

oe=OrdinalEncoder()
for col in train_data:
    train_data[col]=oe.fit_transform(np.asarray(train_data[col].astype('str')).reshape(-1,1))
for col in test_data:
    test_data[col]=oe.fit_transform(np.asarray(test_data[col].astype('str')).reshape(-1,1))

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(y.shape)

In [ ]:
test_data.head(10)

In [ ]:
train_data.head(10)

In [ ]:
## spliting data to X and y


X = train_data.drop(columns = "SalePrice")
print(X.shape)
print(y.shape)
print(test_data.shape)

# ****Scaling data 

In [ ]:
## Scaling Model 
## scale to train_data
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X.shape)


# *****Spliting data to X_train, y_train,X_test and y_test

In [ ]:
## spliting data into X_train , y_train,X_test,y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44, shuffle =True)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# ****apply the RandomForestRegressor Algorithm

In [ ]:
RandomForestRegressorModel = RandomForestRegressor(n_estimators=100,max_depth=5, random_state=44,min_samples_split=5,min_samples_leaf=5
                                                ,n_jobs=-1 )
                                                  
RandomForestRegressorModel.fit(X_train, y_train)


In [ ]:
#Calculating Details
print('Random Forest Regressor Train Score is : ' , RandomForestRegressorModel.score(X_train, y_train))
print('Random Forest Regressor Test Score is : ' , RandomForestRegressorModel.score(X_test, y_test))


In [ ]:
#Calculating Prediction
y_pred =  RandomForestRegressorModel.predict(X_test)

print('Predicted Value for RandomForestRegressorModelis : ' , y_pred)

print(y_pred.shape)

In [ ]:
MAEValue = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
print("MAEValue is :" ,MAEValue)

In [ ]:
MSEValue= mean_squared_error(y_test, y_pred, multioutput='raw_values')
print("MSEValue is : " , MSEValue)

# ****knowing the best features 

In [ ]:
## Knowing the best features 
sel = SelectFromModel(RandomForestRegressor( )) 
sel.fit(X,y)
selected_features = sel.transform(X)
sel.get_support()


In [ ]:
feature_importance =  RandomForestRegressorModel.feature_importances_
feature_importance

# ****Make Predictions

In [ ]:
test_preds = RandomForestRegressorModel.predict(test_data)

In [ ]:
print(test_preds.shape)

In [ ]:
print(test_data)

# ****Submission

In [ ]:
y_pred = np.floor(np.expm1(RandomForestRegressorModel.predict(test_data)))
y_pred

In [ ]:
test_id = test_data['Id']
sub = pd.DataFrame()
sub['Id'] = test_id
sub.to_csv('mysubmission.csv',index=False)